In [4]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode


# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

In [2]:
def sacar_tuplas_imbd(archivo):
    """
    Extrae las tuplas de datos de una archivo CSV.

    Parameters:
    archivo (str): La ruta del archivo CSV.

    Returns:
    list: Una lista de tuplas que contienen los datos extraídos del archivo CSV.
    """

    # Lee el archivo CSV y carga los datos en un DataFrame
    df_detalles = pd.read_csv(archivo)
    
    # Lista para almacenar las tuplas extraídas
    val = []
    
    # Lista para almacenar las filas duplicadas
    duplicadas = []

    # Recorre cada fila del DataFrame
    for pelicula in df_detalles.values:
        
        # Convierte la fila en una lista
        pelicula = list(pelicula)
        
        # Itera sobre cada elemento de la lista
        for i, elemento in enumerate(pelicula):
            
            # Reemplaza los valores NaN con None
            if elemento is np.nan:
                pelicula[i] = None
            
            # Intenta convertir los elementos de tipo str a float o int si es posible
            if isinstance(elemento, str):
                if "." in elemento:
                    try:
                        pelicula[i] = float(elemento)
                    except:
                        pass
                
                if any(caracter.isdigit() for caracter in elemento):
                    try:
                        pelicula[i] = int(elemento)
                    except:
                        pass
                    
                if elemento.startswith('-') and elemento.endswith('-'):
                    try:
                        pelicula[i] = None
                    except:
                        pass
    
        # Verifica si la película ya está en la lista de datos
        for dato in val:
            
            # Si la pelicula ya se encontraba en la lista, la añade a las duplicadas
            if pelicula[-1] in dato:
                duplicadas.append(pelicula)
            
        # Agrega la película a la lista de datos si no está duplicada
        if pelicula not in duplicadas:
            val.append(tuple(pelicula))
        
    # Retorna la lista de tuplas
    return val

In [4]:
# Asigna a la variable el archivo csv que queremos importar a la BBDD

archivo = "BBDD_Importaciones/Actores_detalles/informacion_actores_accion.csv"

In [5]:
# Establece la conexión con la el servidor MySQL y la BBDD

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='proyecto_cinemextract')


# Inserta los datos usando la llamada a la función en la tabla deseada

mycursor = cnx.cursor()
query = "INSERT INTO Actores (Actor, AnioNacimiento, QueHace, ConocidoPor, Premios) VALUES (%s, %s, %s, %s, %s)"
val = sacar_tuplas_imbd(archivo)

try: 
    mycursor.executemany(query, val)
    cnx.commit()
    print(mycursor.rowcount, "registro insertado.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1406 (22001): Data too long for column 'Premios' at row 10
Error Code: 1406
SQLSTATE 22001
Message Data too long for column 'Premios' at row 10


In [20]:
import pandas as pd
import numpy as np

def sacar_tuplas_imbd(archivo):
    """
    Extrae las tuplas de datos de una archivo CSV.

    Parameters:
    archivo (str): La ruta del archivo CSV.

    Returns:
    list: Una lista de tuplas que contienen los datos extraídos del archivo CSV.
    """

    # Lee el archivo CSV y carga los datos en un DataFrame
    df_detalles = pd.read_csv(archivo)
    
    # Lista para almacenar las tuplas extraídas
    val = []
    
    # Recorre cada fila del DataFrame
    for pelicula in df_detalles.values:
        
        # Convierte la fila en una lista
        pelicula = list(pelicula)
        
            
        # Procesa la columna 'Premios'
        premios_str = pelicula[4]  # Obtén la cadena de premios
        if isinstance(premios_str, str):  # Verifica si es una cadena
            premios = eval(premios_str)  # Convierte la cadena en una lista usando eval()
            if premios == []:
                pelicula[4] = None
            elif len(premios) > 3:
                pelicula[4] = str(premios[:3])  # Convierte los primeros 3 premios a cadena
            else:
                pelicula[4] = str(premios)  # Convierte la lista completa a cadena
        else:
            pelicula[4] = None  # Si no es una cadena, establece el valor como None
            
            
        # Procesa la columna 'Anio'
        if pd.isnull(pelicula[1]):  # Verifica si la columna 'Anio' es NaN
            pelicula[1] = None  # Si es NaN, establece el valor como None
                        
        # Reemplaza los valores NaN con None
        for i, elemento in enumerate(pelicula):
            if isinstance(elemento, str) and elemento.strip() == "":
                pelicula[i] = None
                
            # Intenta convertir los elementos de tipo str a float o int si es posible
            if isinstance(elemento, str):
                if "." in elemento:
                    try:
                        pelicula[i] = float(elemento)
                    except:
                        pass
                
                if any(caracter.isdigit() for caracter in elemento):
                    try:
                        pelicula[i] = int(elemento)
                    except:
                        pass
                    
                if elemento.startswith('-') and elemento.endswith('-'):
                    try:
                        pelicula[i] = None
                    except:
                        pass
    
        # Agrega la película a la lista de datos
        val.append(tuple(pelicula))
        
    # Retorna la lista de tuplas
    return val


In [26]:
archivo = "BBDD_Importaciones/Actores_detalles/informacion_actores_drama.csv"

In [27]:
# Establece la conexión con la el servidor MySQL y la BBDD

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='proyecto_cinemextract')


# Inserta los datos usando la llamada a la función en la tabla deseada

mycursor = cnx.cursor()
query = "INSERT INTO Actores (Actor, AnioNacimiento, QueHace, ConocidoPor, Premios) VALUES (%s, %s, %s, %s, %s) ON DUPLICATE KEY UPDATE Actor = VALUES(Actor), AnioNacimiento = VALUES(AnioNacimiento), QueHace = VALUES(QueHace), ConocidoPor = VALUES(ConocidoPor), Premios = VALUES(Premios)"
#query = "INSERT INTO Actores (Actor, AnioNacimiento, QueHace, ConocidoPor, Premios) VALUES (%s, %s, %s, %s, %s)"

val = sacar_tuplas_imbd(archivo)

try: 
    mycursor.executemany(query, val)
    cnx.commit()
    print(mycursor.rowcount, "registro insertado.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

2743 registro insertado.


In [ ]:

# Ejemplo de uso
archivo = "tu_archivo.csv"
val = sacar_tuplas_imbd(archivo)
print(val)  # Imprime la lista de tuplas para verificación
